# Normalization Attention Tutorial

This tutorial works through the 'normalization model of attention' based
on the 2009 paper published in Neuron by John Reynolds and David Heeger.
The model is worth going through because it hits on a range of topics
associated with this course - linear filtering, normalization, population
responses, spatial attention and feature-based attention.  It's powerful
because it can predict a range of physiological, behavioral and fMRI
results from attention studies.

Written by G.M. Boynton in June 2010 based heavily on code provided by the authors.

Translated to Python by Michael Waskom

In [ ]:
%matplotlib inline
import numpy as np
from numpy import pi, sqrt
from numpy.fft import fft, ifft, fftshift
from scipy.stats import norm
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
from ipywidgets import interact

## The 'neural image'

The backbone of the model is the way the population of neuronal responses
are represented.  The 'neural image' is a 2-D matrix (or image) in which
the two matrix dimensions represent two physical dimensions - space and
orientation in this model - with the intensity of each pixel representing
the size of the response to a neuron with the corresponding space and
feature selectivity.   

Showing this matrix as a gray-scale image is a convenient way of
representing the population response for neurons tuned to specific
locations (receptive fields) and orientations.

## Model/neural parameters

First we'll define our 2-d stimulus space for the neural images.  Space
(matrix rows) occupies a single dimension which may seem weird, but
without loss of any generality we can think of the space dimesion as 2-d
space unwrapped into a single vector.  Or, we can just think of it as 1-d
space, since all computations will generalize to 2-d space.  If you
really wanted, you could define a cube, or a 'neural volume' instead of a
neural image with 2 dimension for space and the third dimension as the
feature. (and so on).

The dictionary `p` will eventually contain all parameters of the model.

In [ ]:
p = dict(
    x=np.arange(-200, 204, 4),
    theta=np.arange(-180, 181),
)

## Stimulus parameters

Stimuli will be defined as having centers and Gassian widths in both
space and orientation.  These are basically like Gabors if the
orientation width is narrow and if we assume that the stimulus is
narrowly defined in spatial frequency.

We'll define the stimulus parameters inside the dictionary 'stim'.  Each
of the fields contain vectors of equal length, with each component
corresponding to a different stimulus.  We'll start with a single
stimulus - a high-contrast Gabor at position `x = -100` with an orientation
of `theta = 0`.

In [ ]:
stim = dict(
    x_center=[-100],  # stimulus center_positions
    x_width = [3],  # stimulus widths (deg)
    th_center=[0],  # stimulus orientation centers
    th_width=[1],  # stimulus orientation widths
    contrast=[1],
)

A stimulus can be represented as a 'stimulus image' in the same
coordinates as the neural image.  Each pixel in the stimulus image
represents the amount of contrast at that pixel's corresponding position
and orientation.

We can translate these stimulus parameters into a stimulus image with the
following function. It works by generating Gaussians in space and orientation
for each stimulus component and computing the outer product of the two to generate
the stimulus image matrix.  So the contrast energy of the stimulus is defined to
be a separable matrix with Gaussian orientation and spatial tuning on the marginals.

In [ ]:
def make_neural_image(p, stim):
    """Make a 2D array representing the population response."""
    stim = stim.copy()
    stim.setdefault("contrast", [1] * len(stim["x_center"]))
    stim.setdefault("rng", (0, 1))
    stim.setdefault("method", "multiply")
    
    img = np.zeros((p["theta"].size, p["x"].size))
    for i, _ in enumerate(stim["x_center"]):

        x_w_i, x_c_i = stim["x_width"][i], stim["x_center"][i]
        th_w_i, th_c_i = stim["th_width"][i], stim["th_center"][i]
        
        x_g = x_w_i * sqrt(2 * pi) * norm.pdf(p["x"], x_c_i, x_w_i)
        th_g = th_w_i * sqrt(2 * pi) * norm.pdf(p["theta"], th_c_i, th_w_i)

        if stim["method"] == "multiply":
            sub_img = np.outer(th_g, x_g)
        elif stim["method"] == "add":
            sub_img = np.average(np.meshgrid(x_g, th_g), axis=0)

        img += stim["contrast"][i] * sub_img

    rng = stim["rng"]
    img = img * (rng[1] - rng[0]) + rng[0]
    return img

def show_neural_image(p, img, **kws):
    """Plot a 2D population response image as a heatmap."""
    f, ax = plt.subplots(figsize=(7, 6))
    ax.pcolormesh(p["x"], p["theta"], img, cmap="gray", **kws)
    ax.set(xlabel="Position", ylabel="Orientation",
           yticks=[-180, -90, 0, 90, 180])

In [ ]:
show_neural_image(p, make_neural_image(p, stim))

Take some time to think about what different stimuli look like in this
stimulus space. What does a full-field grating look like?  It's a
horizontal stripe.  A spatially localized patch of noise?  A vertical
stripe.  (Note that a 'stimulus image' here is different from an actual
image of the stimulus)

## Excitatory response parameters (neuronal tuning widths)

The first stage of the model is the linear excitatory response to the
stimulus.  This is simply the response of a linear receptive field tuned
to space and feature.  The center of the tuning for each neuron is
determined by where it lives in the neural image.  The width of tuning
for space (receptive field size) and feature (orientation tuning) are
defined here.  These are in terms of standard deviations of Gaussians.

In [ ]:
p["e"] = dict(x_width=5, th_width=60)

## The Excitatory response

The neural response for the excitatory component is a linear filtering of
the stimulus with the excitatory receptive fields. For any given pixel in
the neural image, the response is a Gaussian centered at that position
with widths determined by p.e above, multiplied by the stimulus image,
and added up.  The entire neural image is therefore calculated with a
convolution of the stimulus image by the Gaussians determined by the
excitatory parameters.  

We'll define a function `convolve_image` that produces this neural image
through convolution:

In [ ]:
def convolve_image(p, img, x_width, theta_width):

    dx = p["x"][1] - p["x"][0]
    dtheta = p["theta"][1] - p["theta"][0]

    # Standard convolution for space (x) dimension
    x_c = p["x"][p["x"].size // 2]
    x_filt = norm.pdf(p["x"], x_c, x_width)
    out = dx * convolve2d(img, x_filt[np.newaxis, :], "same")

    # Circular convolution for orientation (theta) dimension
    theta_c = p["theta"][p["theta"].size // 2]
    theta_filt = norm.pdf(p["theta"], theta_c, theta_width)
    theta_filt = np.tile(theta_filt[:, np.newaxis], len(p["x"]))

    conv_fft = fft(out, axis=0) * fft(theta_filt, axis=0)
    out = dtheta * fftshift(ifft(conv_fft, axis=0).real, axes=0)

    return out

Here's what the excitatory neural image looks like (the authors call it 'excitatory drive')

In [ ]:
img = make_neural_image(p, stim)
E = convolve_image(p, img, p["e"]["x_width"], p["e"]["th_width"])
show_neural_image(p, E)

Does this make sense?  Remember, the intensity of the image at each point
is the response of a neuron with tuning centered at the corresponding
location and orientation.  The orientation tuning with is pretty broad
(60 deg), so neurons tuned fairly far away from the stimulus still
respond somewhat to the stimulus.  But the spatial receptive field is
narrow (5 deg), so the neural image drops off rapidly in the spatial (x)
dimension.

## Inhibitory response parameters

In the standard normalization model, the excitatory signal is divided by
the pooled response across a range of neurons tuned across space and
features.  We can define the range of spatial pooling the same way we
defined the stimulus image and the excitatory response parameters:


In [ ]:
p["i"] = dict(x_width=20, th_width=360)

We also need a constant in the denominator to keep the ratio from blowing up:

In [ ]:
p["sigma"] = 1e-6

These specific parameters mean that each neuron is suppressed by neurons
tuned away across a fairly narrow range in space, but across neurons
tuned across all orientations.  

The neural image describing the summed response in the pool of neurons is
therefore simply the convolution of the excitatory neural image (E) with
Gaussians determined by the inhibitory pooling parameters.

In [ ]:
I = convolve_image(p, E, p["i"]["x_width"], p["i"]["th_width"])

Here's a picture of the inhibitory pooling neural image which the authors call 'inhibitory drive':

In [ ]:
show_neural_image(p, I)

The brightness of each pixel in this image represents the strength of
divisive inhibition for the corresponding neuron.

Although these inhibitory parameters are defined the same way as the
excitatory parameters, they have a very different meaning.  The
excitatory parameters determine the receptive field properties of the
neurons.  These inhibitory parameters determine the range of neurons that
we're polling across for normalization. It's similiar in that the
excitatory parameters determine the range of stimuli excite a given
neuron, the inhibitory parameters determine the range of neurons that
inhibit a given neuron.

The neural image for the normalized response is calculated as:

In [ ]:
R = E / (I + p["sigma"])
show_neural_image(p, R)

This is the population response for the normalization model.  As
expected, there are the largest responses for neurons tuned for the
stimulus properties, and the neural responses fall of for neurons tuned
away for both space and orientation.  

You should play with the model parameters to see how they affect is
neural image:

In [ ]:
@interact
def response_tutorial(e_x_width=(0, 10), i_x_width=(10, 30),
                      e_theta_width=(30, 90), i_theta_width=(90, 360)):

    S = make_neural_image(p, stim)
    E = convolve_image(p, img, e_x_width, e_theta_width)
    I = convolve_image(p, E, i_x_width, i_theta_width)
    R = E / (I + p["sigma"])
    show_neural_image(p, R)

Before we get in to attention, we can play around a bit with this basic
normalization model to see how it predicts some basic response properties
of V1 neurons.


## Contrast response

Let's measure the response of the normalization model to a range of
contrasts.  This means generating a neural image for each contrast. We'll
define a function `normalization_model` that calculates the neural image, `R`,
shown above (and returns the other images for free)


In [ ]:
def normalization_model(p, stim, attend=None, return_all=False):

    S = make_neural_image(p, stim)
    E = convolve_image(p, S, p["e"]["x_width"], p["e"]["th_width"])

    if attend is not None:
        A = make_neural_image(p, attend)
    else:
        A = np.ones_like(E)

    G = E * A
    I = convolve_image(p, G, p["i"]["x_width"], p["i"]["th_width"])
    R = G / (I + p["sigma"])

    if return_all:
        return R, S, E, A, G, I
    return R

In [ ]:
contrasts = np.logspace(-12, 0, 21, base=np.e)
R = np.zeros((contrasts.size, p["theta"].size, p["x"].size))
for i, c in enumerate(contrasts):
    stim["contrast"] = [c]
    R[i] = normalization_model(p, stim)

Most of the neurons in these neural images are not responding because
they are tuned away from the stimulus in space or orientation. Let's find
the neuron that is most closely tuned to the center of the stimulus and
plot its contrast response function.

In [ ]:
x_idx = np.argmin((p["x"] - stim["x_center"]) ** 2)
th_idx = np.argmin((p["theta"] - stim["th_center"]) ** 2)

In [ ]:
f, ax = plt.subplots(figsize=(5, 5))
ax.plot(np.log(contrasts), R[:, th_idx, x_idx], label="Grating")
ax.set(xlabel="log(contrast)", ylabel="Response")
f.tight_layout()

The value of sigma in the denominator of the normalization equation controls the contrast gain:

In [ ]:
@interact
def contrast_gain(sigma_exp=(-8, -4, .5)):

    contrasts = np.logspace(-12, 0, 21, base=np.e)
    p["sigma"] = 10 ** sigma_exp

    R = np.zeros((contrasts.size, p["theta"].size, p["x"].size))
    for i, c in enumerate(contrasts):
        stim["contrast"] = [c]
        R[i] = normalization_model(p, stim)

    f, ax = plt.subplots(figsize=(5, 5))
    ax.plot(np.log(contrasts), R[:, th_idx, x_idx], label="Grating")
    ax.set(xlabel="log(contrast)", ylabel="Response", ylim=(-1, 23))
    f.tight_layout()

## Cross-orientation inhibition

Consider the contrast response for the same neuron, but in the presence
of a high contrast grating tuned to the orthognal dimension.  This is a
sequence of plaids where the off-orientation is high contrast and the
preferred orientation varies:

In [ ]:
stim.update(
    x_center=[-100, -100],
    x_width=[3, 3],
    th_center=[0, 90],
    th_width=[1, 1],
)

p["sigma"] = 1e-6

In [ ]:
contrasts = np.logspace(-12, 0, 21, base=np.e)
R = np.zeros((contrasts.size, p["theta"].size, p["x"].size))
for i, c in enumerate(contrasts):
    stim["contrast"] = [c, 1]
    R[i] = normalization_model(p, stim)

In [ ]:
ax.plot(np.log(contrasts), R[:, th_idx, x_idx], label="Plaid")
ax.legend(loc="upper left")
f

Notice how the presence of a high contrast orthogonal stimulus (orange)
suppresses the responses to the stimulus compared to the grating alone
(blue).  This is exactly what's found in the physiology literature and
has been modeled by normalization models such as Heeger's work in the
90's. Technically, Heeger's original model squares the excitatory input
before feeding into the normalization process, but the general idea is
the same.

The normalization model predicts these curves because the normalization
pool is orientation-independent, so the orthogonal grating adds a strong
divisive signal.

You probably noticed that the response to the plaid is greater than the
response to the grating at low grating contrasts.  This is because the
excitatory orientation tuning is broad (60 deg), so the orthogonal
stimulus feeds some signal into the excitatory input.

## Stimulus parameters for an attention experiment

Finally we're ready to talk about attention.  First we'll set up a new
stimulus condition.  This will be a classic spatial attention condition
where two intermediate contrast Gabors are presented, one on the left and
one on the right side of the visual field.

In [ ]:
stim = dict(
    x_center=[-100, 100],
    x_width=[3, 3],
    th_center=[0, 0],
    th_width=[1, 1],
    contrast=[.25, .25],
)

Here's the stimulus image

In [ ]:
S = make_neural_image(p, stim)
show_neural_image(p, S)

## Attention parameters

The normalization model of attention defines the spatial and featural
spread of attention using the same sort of structure as the stimulus,
excitatory and inhibitory parameters.  

Spatial attention is a Gaussian 'spotlight' centered at some location
with some standard deviation that may vary with the task. We'll have 
attention directed to the left stimulus (`x = 100`) and focused down to a
size that matches the size of the stimulus (`width = 3`):

In [ ]:
attend = dict(
    x_center=[-100],
    x_width=[3],
)

Feature-based attention is defined similarly with a center and a width.

Here's an example of feature-based attention being spread across all
orientations using a very large `th_width`.

In [ ]:
attend.update(
    th_center=[0],
    th_width=[1e3],
)

The final attention parameters determine the minimum and maximum gain changes.  

In [ ]:
attend.update(rng=(1, 2))

The 'spotlight' of attention can be represented as an 'Attention Field' in the neural image space and can be generated with 'makeNeuralImage'

In [ ]:
A = make_neural_image(p, attend)
show_neural_image(p, A)

## Modelling attention by modulating the excitatory input

The effects of attention are implemented by simply multiplying the
excitatory drive by the attentional spotlight. 

Here's the excitatory input, as before:

In [ ]:
E = convolve_image(p, S, p["e"]["x_width"], p["e"]["th_width"])

And here's the gain change due to attention:

In [ ]:
G = A * E
show_neural_image(p, G)

See how the excitatory response to the left stimulus is greater (brighter)
than the right, because attention was directed there.

## Inhibitory image with attention

Remakably, the rest of the model is exactly as before.  We calculate the
inhibitory neural image by convolving the (now modulated) excitatory
input:

In [ ]:
I = convolve_image(p, G, p["i"]["x_width"], p["i"]["th_width"])
show_neural_image(p, I)

Notice that for this example, the divisive inhibitory input is also
greater for the attended stimulus. We'll see soon how the relative
contributions of attention to the excitatory and inhibitory inputs allows
for a range of ways that attention can influence the neuronal response.

## Normalization model with attention

The last step is also like before, we divide the response by the
inhibitory input (plus a small constant).

In [ ]:
R = G / (I + p["sigma"])
show_neural_image(p, R)

## Contrast response for attended and unattended stimuli

The way attention influences neuronal responses as a function of stimulus
contrast is a useful way to characterize the overall effects of
attention.

Next we'll plot contrast response functions for neurons that were
presented identical physical stimuli, but with attention directed within
only one of the two receptive fields.  The responses of these two neurons
is equivalent to the response of a single neuron with attention shifted
within and away from its receptive field.  

To plot the contrast response functions, we need to find the indices for
the two neurons that are most closely tuned to the two stimuli:


In [ ]:
x_idxs = np.argmin((p["x"][:, np.newaxis] - stim["x_center"]) ** 2, axis=0)
th_idxs = np.argmin((p["theta"][:, np.newaxis] - stim["th_center"]) ** 2, axis=0)

Now we can calculate the neural images and plot the two responses just
like before.  The function 'normalizationModel' can take in a third
argument 'attend' that contains the attention parameters.  The
calculations within 'normalizationModel' are identical to those in this
script.

In [ ]:
R = np.zeros((contrasts.size, p["theta"].size, p["x"].size))
for i, c in enumerate(contrasts):
    stim["contrast"] = [c, c]
    R[i] = normalization_model(p, stim, attend)

We can pull out the two contrast response functions to each stimulus from the neural images:

In [ ]:
y = R[:, th_idxs, x_idxs]
f, ax = plt.subplots(figsize=(5, 5))
ax.plot(np.log(contrasts), y[:, 0], label="Attend in")
ax.plot(np.log(contrasts), y[:, 1], label="Attend out")
ax.set(xlabel="log(contrast)", ylabel="Response")
ax.legend(loc="upper left")
f.tight_layout()

There it is, a higher response to an attended stimulus than an unattended
stimulus.  Note that for these parameters, it looks like attention is
acting as a 'response gain', which is a vertical scaling between the
attended and unattended responses across contrast.  

## Response gain vs. Contrast gain.

Why does a narrow focus of attention lead to response gain?  It helps to
look at the cross-section of the neural images to get some insight.
Consider a high-contrast stimulus with a narrow focus of attention. We'll
make the conditions more extreme (larger stimulus, smaller focus of
attention) for better illustration.  Here is a complete set of stimulus
and attentional parameters:


In [ ]:
stim = dict(
    x_center=[-100, 100],
    x_width=[20, 20],
    th_center=[0, 0],
    th_width=[1, 1],
    contrast=[1, 1],
)

attend = dict(
    x_center=[-100],
    x_width=[1],
    th_center=[0],
    th_width=[1e3],
    rng=(1, 2),
)

In [ ]:
R, S, E, A, G, I = normalization_model(p, stim, attend, return_all=True)

Here's a plot of the spatial profile of the excitatory drive and inhibitory drives after it is modulated by attention (G).

In [ ]:
idx = p["theta"].size // 2
f, (ax_g, ax_i) = plt.subplots(2, 1, sharex=True)
ax_g.plot(p["x"], G[idx])
ax_g.set(title="Excitatory drive")
ax_i.plot(p["x"], I[idx])
ax_i.set(xlabel="Space", title="Inhibitory drive")
f.tight_layout()

You can see how the effect of attention on the excitatory drive is to
multiply the original excitatory input by a narrow Gaussian (ranging
between 1 and 2), which increases the excitatory drive only within cells
with RF's near the attended location (like the one we're plotted from
above).

The inhibitory drive is the convolution of the excitatory drive by a
pooling filter, which simply spatially blurrs the excitatory drive. Since
the spike is so narrow, the spatial blurring by the pooling process
leaves the inhibitory drive very similar for the attended and unattened
regions of space (left vs. right).

The output of the model is basically the ratio of the excitatory and the
inhibitory drives.  Since convolution is linear, changing the contrast
simply scales these curves up and down (try running this section again
with a different contrast.  Only the y-axis scales). So you can see how
for the neurons with RF's centered at the focus of attention, the effect
of attention is all in the numerator.  So scaling by contrast simply
scales the response.  Contrast gain!

Now, consider the same stimulus but with a broad focus of spatial attention:

In [ ]:
attend["x_width"] = [30]
R, S, E, A, G, I = normalization_model(p, stim, attend, return_all=True)

ax_g.plot(p["x"], G[idx], ls="--")
ax_i.plot(p["x"], I[idx, ], ls="--")
ax_g.legend(["Narrow focus", "Broad focus"])
f

The effect of a broad focus of spatial attention is to boost a broader 
neuronal population since we multiplied by a broader Gaussian.  Note,
however, that the peak excitatory drive is the same as for the narrow
focus. 

This time, the inhibitory drive is strongly affected by attention.  This
is because the blurring of the new excitatory drive by the attention
filter is summing the response over a lot of active neurons. 

As before, contrast simply scales both the excitatory and inhibitory
drives up and down.  But now both the numerator AND denominator for the
model are growing faster with contrast for the 'attended' neuron.  This
is just like changing the contrast of the stimulus with attention.
Contrast gain!  

Reynolds and Heeger go and predict a range of published papers showing
both contrast gain and response gain effects of attention just by
changing the spatial focus of attention relative to the size of the
stimulus. This helps to settle a lot of discrepancies (and debates) in
the literature.  This simple explanation also leads to some testable
hypotheses about how the spatial focus of attention should affect
neronal, behavioral and fMRI measurements.

## Attention and orientation tuning.

McAdams and Maunsell (1999) measured how the orientation tuning of V4
neurons are affected by spatial attention by measuring orientation tuning
while monkeys attended either inside or outside the receptive field of
each neuron. 

The model naturally predicts how spatial and feature-based attention
influences orientation tuning curves.  We can see this by simply slicing
through the neural images along the orientation dimension.  For
simplicity, we'll assume that as before, while feature-based attention is
localized, attention is directed to all orientations.

Let's set up some reasonable stimulus and attention conditions:

In [ ]:
stim = dict(
    x_center=[-100, 100],
    x_width=[10, 10],
    th_center=[0, 0],
    th_width=[1, 1],
    contrast=[1, 1],
)

attend = dict(
    x_center=[-100],
    x_width=[10],
    th_center=[0],
    th_width=[1e3],
    rng=(1, 4),
)

R = normalization_model(p, stim, attend)

f, ax = plt.subplots()
ax.plot(p["theta"], R[:, x_idxs[0]])
ax.plot(p["theta"], R[:, x_idxs[1]])
ax.set(xlabel="Orientation (deg)")
ax.legend(["Attended", "Unattended"], loc="upper left")
f.tight_layout()

This looks just like the scaling of the tuning functions seen by McAdams and Maunsell
(1999).  

You might think it's weird that we're plotting slices of the neural image
which is really the model's prediction of the response of a bunch of
different neurons to the same stimulus.  But if you think about it, this
is just the same as plotting the response of the same neuron to a range
of stimuli.  

The model (as I've implemented it) used this 'or' rule all along but it
wasn't noticeable because `attend["th_width"]` is very large , so 'and'
and 'or' predict the same thing.

But with `attend["th_width"] = 30`, we can now look at the effects of
feature based attention.

## Moran and Desimone

The study by Moran and Desimone (1985) was the first to show attentional
effects in monkey cortex.  V4 neurons were measured with two oriented bars 
in the receptive field.



In [ ]:
stim = dict(
    x_center=[-100, -100],
    x_width=[10, 10],
    th_center=[0, 90],
    th_width=[1, 1],
    contrast=[1, 1],
)

attend = dict(
    x_center=[-100],
    x_width=[2],
    th_center=[0],
    th_width=[30],
    rng=(1, 2),
    method="add",
)

Rpref = normalization_model(p, stim, attend)
Rpref, Spref, Epref, Apref, Gpref, Ipref = normalization_model(p, stim, attend, return_all=True)

attend["th_center"] = [90]
Rnonpref = normalization_model(p, stim, attend)

y1 = Rpref[th_idxs[0], x_idxs[0]]
y2 = Rnonpref[th_idxs[0], x_idxs[0]]

f, ax = plt.subplots(figsize=(5, 5))
ax.bar(["Attend\nPref", "Attend\nNon-pref"], [y1, y2])
ax.set(ylabel="Model response")
f.tight_layout()

## Feature-similarity gain

This 'cross' profile of attentional gain means that feature-based
attention should influence responses for neurons with receptive fields
outside the focus of spatial attention.  This turns out to be true.
Treue and Martinez-Trujillo (1999) originally discovered this by
measuring the response to an unattended moving stimulus in MT when
feature-based attention was altered by having the monkey perform a
task on a stimulus in the opposite hemfield.  

We can demonstrate this with the model by presenting stimuli that simulate
the conditions for an fMRI experiment by Saenz et al. (2002).

In [ ]:
stim = dict(
    x_center=[-100, -100, 100],
    x_width=[10, 10, 10],
    th_center=[0, 90, 0],
    th_width=[2, 2, 2],
    contrast=[1, 1, 1],
)

attend = dict(
    x_center=[-100],
    x_width=[2],
    th_center=[0],
    th_width=[30],
    rng=(1, 2),
    method="add",
)

Rpref, S, E, Apref, G, I = normalization_model(p, stim, attend, return_all=True)

show_neural_image(p, S)

In the real experiment, the stimuli were moving dots instead of oriented
stimuli but it doesn't matter here.  On the attended (left) side, two
stimuli were presented overlapping in space - one vertical and one
horizontal.  The unattended side contained a single oriented stimulus.

Save the response to the unattended stimulus for the neuron that both
prefers that stimulus and the attended orientation (0).

In [ ]:
yP = Rpref[th_idxs[1], x_idxs[1]]

Now we'll shift feature-based attention to the non-preferred component of the attended stimulus.

In [ ]:
attend["th_center"] = [90]
Rnonpref = normalization_model(p, stim, attend)

Pull out the same neuron's response and plot both:

In [ ]:
yNP = Rnonpref[th_idxs[1], x_idxs[1]]
f, ax = plt.subplots(figsize=(5, 5))
ax.bar(["Attend out\npreferred", "Attend out\nnon-preferred"], [yP, yNP])
ax.set(ylabel="Model response")
f.tight_layout()